# UV-Vis Layer Stack Simulation
## Interactive Transfer Matrix Method (TMM) Calculator

This notebook simulates optical properties (Reflection, Transmission, Absorption) of multilayer thin film stacks using the Transfer Matrix Method.

Based on the MATLAB code by Malte Langenhorst et al. (KIT)

In [19]:
# Import required libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded successfully!")

Libraries loaded successfully!


## Core TMM Functions

These functions implement the Transfer Matrix Method for calculating optical properties of multilayer stacks.

In [20]:
def is_forward_angle(n, theta):
    """
    Check if a wave traveling at angle theta in medium with index n is forward-traveling.
    
    Parameters:
    n: complex refractive index (can be array)
    theta: propagation angle in degrees (can be array)
    
    Returns:
    Boolean array indicating forward direction
    """
    if np.max(np.real(n) * np.imag(n) < 0):
        raise ValueError("For materials with gain, it's ambiguous which beam is incoming vs outgoing.")
    
    if np.isscalar(theta):
        theta = np.ones_like(n) * theta
    
    rad_theta = theta * np.pi / 180.0
    ncostheta = n * np.cos(rad_theta)
    answer = np.zeros(n.shape, dtype=bool)
    
    # Evanescent decay or lossy medium
    evanescent = np.abs(np.imag(ncostheta)) > 100 * np.finfo(float).eps
    answer[evanescent] = np.imag(ncostheta[evanescent]) > 0
    
    # Forward is the one with positive Poynting vector
    propagating = ~evanescent
    answer[propagating] = np.real(ncostheta[propagating]) > 0
    
    return answer


def list_snell(n_list, th_0):
    """
    Apply Snell's law to get angle in each layer.
    
    Parameters:
    n_list: array of refractive indices [wavelengths x layers]
    th_0: incident angle in degrees
    
    Returns:
    angles: array of angles in each layer [wavelengths x layers]
    """
    num_lambdas, num_layers = n_list.shape
    angles = np.zeros((num_lambdas, num_layers), dtype=complex)
    angles[:, 0] = th_0
    
    for i in range(1, num_layers):
        sin_th = n_list[:, 0] * np.sin(th_0 * np.pi / 180.0) / n_list[:, i]
        # Handle complex angles - use np.arcsin which handles complex numbers
        angles[:, i] = np.arcsin(sin_th) * 180.0 / np.pi
    
    return angles


def interface_r(pol, n_i, n_f, th_i, th_f):
    """
    Reflection amplitude at interface between two media.
    
    Parameters:
    pol: polarization ('TE' or 'TM')
    n_i, n_f: refractive indices of incident and final media
    th_i, th_f: angles in incident and final media (degrees)
    
    Returns:
    r: reflection amplitude
    """
    rad_th_i = th_i * np.pi / 180.0
    rad_th_f = th_f * np.pi / 180.0
    if pol == 'TE':
        r = ((n_i * np.cos(rad_th_i) - n_f * np.cos(rad_th_f)) /
             (n_i * np.cos(rad_th_i) + n_f * np.cos(rad_th_f)))
    elif pol == 'TM':
        r = ((n_f * np.cos(rad_th_i) - n_i * np.cos(rad_th_f)) /
             (n_f * np.cos(rad_th_i) + n_i * np.cos(rad_th_f)))
    else:
        raise ValueError("Polarization must be 'TE' or 'TM'")
    
    return r


def interface_t(pol, n_i, n_f, th_i, th_f):
    """
    Transmission amplitude at interface between two media.
    
    Parameters:
    pol: polarization ('TE' or 'TM')
    n_i, n_f: refractive indices of incident and final media
    th_i, th_f: angles in incident and final media (degrees)
    
    Returns:
    t: transmission amplitude
    """
    rad_th_i = th_i * np.pi / 180.0
    rad_th_f = th_f * np.pi / 180.0
    if pol == 'TE':
        t = (2 * n_i * np.cos(rad_th_i) /
             (n_i * np.cos(rad_th_i) + n_f * np.cos(rad_th_f)))
    elif pol == 'TM':
        t = (2 * n_i * np.cos(rad_th_i) /
             (n_f * np.cos(rad_th_i) + n_i * np.cos(rad_th_f)))
    else:
        raise ValueError("Polarization must be 'TE' or 'TM'")
    
    return t


def R_from_r(r):
    """Calculate reflectance from reflection amplitude."""
    return np.abs(r * r)


def T_from_t(pol, t, n_i, n_f, th_i, th_f):
    """
    Calculate transmittance from transmission amplitude.
    
    Parameters:
    pol: polarization ('TE' or 'TM')
    t: transmission amplitude
    n_i, n_f: refractive indices
    th_i, th_f: angles (degrees)
    
    Returns:
    T: transmittance
    """
    rad_th_i = th_i * np.pi / 180.0
    rad_th_f = th_f * np.pi / 180.0
    if pol == 'TE':
        T = (np.abs(t * t) * 
             (np.real(n_f * np.cos(rad_th_f)) / 
              np.real(n_i * np.cos(rad_th_i))))
    elif pol == 'TM':
        T = (np.abs(t * t) * 
             (np.real(n_f * np.conj(np.cos(rad_th_f))) / 
              np.real(n_i * np.conj(np.cos(rad_th_i)))))
    else:
        raise ValueError("Polarization must be 'TE' or 'TM'")
    
    return T


def power_entering_from_r(pol, r, n_i, th_i):
    """
    Calculate power entering the first interface from reflection amplitude.
    
    Parameters:
    pol: polarization
    r: reflection amplitude
    n_i: incident medium refractive index
    th_i: incident angle (degrees)
    
    Returns:
    Power entering (usually 1-R)
    """
    rad_th_i = th_i * np.pi / 180.0
    if pol == 'TE':
        P_in = (np.real(n_i * np.cos(rad_th_i) * (1 + np.conj(r)) * (1 - r)) /
                np.real(n_i * np.cos(rad_th_i)))
    elif pol == 'TM':
        P_in = (np.real(n_i * np.conj(np.cos(rad_th_i)) * (1 + r) * (1 - np.conj(r))) /
                np.real(n_i * np.conj(np.cos(rad_th_i))))
    else:
        raise ValueError("Polarization must be 'TE' or 'TM'")
    
    return P_in

print("Helper functions defined successfully!")

Helper functions defined successfully!


In [21]:
def coh_tmm(pol, n_array, d_list, th_0, lam_vac):
    """
    Main coherent transfer matrix method calculation.
    
    Parameters:
    pol: polarization ('TE' or 'TM')
    n_array: refractive indices [wavelengths x layers]
    d_list: layer thicknesses (first and last should be inf)
    th_0: incident angle (degrees)
    lam_vac: vacuum wavelengths
    
    Returns:
    Dictionary with R, T, A, and other optical properties
    """
    # Input validation
    if np.ndim(th_0) > 0 and len(th_0) > 1:
        raise ValueError('This function is not vectorized for multiple angles')
    
    num_layers = len(d_list)
    num_lambdas = len(lam_vac)
    
    if d_list[0] != np.inf or d_list[-1] != np.inf:
        raise ValueError('d_list must start and end with inf!')
    
    # Make n_array real for incident medium
    n_array[:, 0] = np.real(n_array[:, 0])
    
    # Calculate angles in each layer using Snell's law
    th_array = list_snell(n_array, th_0)
    rad_th_array = th_array * np.pi / 180.0
    
    # Calculate z-component of wavevector
    kz_array = 2 * np.pi * n_array * np.cos(rad_th_array) / lam_vac[:, np.newaxis]
    
    # Calculate phase accumulated in each layer
    delta = kz_array * d_list
    
    # For very opaque layers, reset delta to avoid numerical errors
    delta[np.imag(delta) > 35] = np.imag(delta[np.imag(delta) > 35]) * 1j
    
    # Initialize transfer matrices
    t_list = np.zeros((num_lambdas, num_layers), dtype=complex)
    r_list = np.zeros((num_lambdas, num_layers), dtype=complex)
    
    for i in range(num_layers - 1):
        t_list[:, i] = interface_t(pol, n_array[:, i], n_array[:, i+1], 
                                   th_array[:, i], th_array[:, i+1])
        r_list[:, i] = interface_r(pol, n_array[:, i], n_array[:, i+1],
                                   th_array[:, i], th_array[:, i+1])
    
    # Build transfer matrix
    M_list = np.zeros((num_lambdas, 2, 2), dtype=complex)
    M_list[:, 0, 0] = 1
    M_list[:, 1, 1] = 1
    
    for i in range(1, num_layers - 1):
        M_i = np.zeros((num_lambdas, 2, 2), dtype=complex)
        M_i[:, 0, 0] = np.exp(-1j * delta[:, i])
        M_i[:, 1, 1] = np.exp(1j * delta[:, i])
        
        # Interface matrix
        M_interface = np.zeros((num_lambdas, 2, 2), dtype=complex)
        M_interface[:, 0, 0] = 1 / t_list[:, i]
        M_interface[:, 0, 1] = r_list[:, i] / t_list[:, i]
        M_interface[:, 1, 0] = r_list[:, i] / t_list[:, i]
        M_interface[:, 1, 1] = 1 / t_list[:, i]
        
        # Multiply matrices
        M_temp = np.matmul(M_i, M_interface)
        M_list = np.matmul(M_list, M_temp)
    
    # Calculate reflection and transmission coefficients
    M_tot = M_list
    r = M_tot[:, 1, 0] / M_tot[:, 0, 0]
    t = 1 / M_tot[:, 0, 0]
    
    # Calculate R and T
    R = R_from_r(r)
    T = T_from_t(pol, t, n_array[:, 0], n_array[:, -1], 
                 th_array[:, 0], th_array[:, -1])
    
    # Power entering
    power_entering = power_entering_from_r(pol, r, n_array[:, 0], th_array[:, 0])
    
    # Calculate forward and backward amplitudes (v, w) in each layer
    vw_array = np.zeros((num_layers, 2, num_lambdas), dtype=complex)
    vw = np.zeros((2, num_lambdas), dtype=complex)
    vw[0, :] = t
    vw[1, :] = 0
    vw_array[-1, :, :] = vw
    
    for i in range(num_layers - 2, 0, -1):
        # Propagation matrix
        M_prop = np.zeros((num_lambdas, 2, 2), dtype=complex)
        M_prop[:, 0, 0] = np.exp(1j * delta[:, i])
        M_prop[:, 1, 1] = np.exp(-1j * delta[:, i])
        
        # Interface matrix (inverse)
        M_int_inv = np.zeros((num_lambdas, 2, 2), dtype=complex)
        M_int_inv[:, 0, 0] = t_list[:, i]
        M_int_inv[:, 0, 1] = -r_list[:, i]
        M_int_inv[:, 1, 0] = -r_list[:, i]
        M_int_inv[:, 1, 1] = t_list[:, i]
        
        vw_new = np.zeros((2, num_lambdas), dtype=complex)
        for lam_idx in range(num_lambdas):
            vw_new[:, lam_idx] = M_prop[lam_idx] @ M_int_inv[lam_idx] @ vw[:, lam_idx]
        vw = vw_new
        vw_array[i, :, :] = vw
    
    # Return results
    return {
        'r': r,
        't': t,
        'R': R,
        'T': T,
        'power_entering': power_entering,
        'vw_array': vw_array,
        'kz_array': kz_array,
        'th_array': th_array,
        'pol': pol,
        'n_array': n_array,
        'd_list': d_list,
        'th_0': th_0,
        'lam_vac': lam_vac
    }

print("Main TMM function defined successfully!")

Main TMM function defined successfully!


In [22]:
def find_in_structure_with_inf(d_list, dist):
    """
    Find which layer a given distance corresponds to.
    
    Parameters:
    d_list: list of thicknesses [inf, ..., inf]
    dist: distance from front of structure
    
    Returns:
    (layer, distance_in_layer)
    """
    if dist < 0:
        return 1, dist
    
    cumsum = 0
    for i in range(1, len(d_list) - 1):
        if cumsum <= dist < cumsum + d_list[i]:
            return i, dist - cumsum
        cumsum += d_list[i]
    
    return len(d_list) - 1, dist - cumsum


def position_resolved(layer, distance, coh_tmm_data):
    """
    Calculate Poynting vector and absorption at specific position.
    
    Parameters:
    layer: layer index
    distance: distance within layer
    coh_tmm_data: output from coh_tmm()
    
    Returns:
    Dictionary with poyn, absor, Ex, Ey, Ez
    """
    num_lambdas = len(coh_tmm_data['lam_vac'])
    
    if layer > 0:
        v = coh_tmm_data['vw_array'][layer, 0, :]
        w = coh_tmm_data['vw_array'][layer, 1, :]
    else:
        v = np.ones(num_lambdas)
        w = coh_tmm_data['r']
    
    kz = coh_tmm_data['kz_array'][:, layer]
    th = coh_tmm_data['th_array'][:, layer]
    n = coh_tmm_data['n_array'][:, layer]
    n_0 = coh_tmm_data['n_array'][:, 0]
    th_0 = coh_tmm_data['th_0']
    pol = coh_tmm_data['pol']
    
    rad_th = th * np.pi / 180.0
    rad_th0 = th_0 * np.pi / 180.0
    
    # Amplitude of forward and backward waves
    Ef = v * np.exp(1j * kz * distance)
    Eb = w * np.exp(-1j * kz * distance)
    
    # Poynting vector
    if pol == 'TE':
        poyn = (np.real(n * np.cos(rad_th) * np.conj(Ef + Eb) * (Ef - Eb)) /
                np.real(n_0 * np.cos(rad_th0)))
    elif pol == 'TM':
        poyn = (np.real(n * np.conj(np.cos(rad_th)) * (Ef + Eb) * np.conj(Ef - Eb)) /
                np.real(n_0 * np.conj(np.cos(rad_th0))))
    
    # Absorbed energy density
    if pol == 'TE':
        absor = (np.imag(n * np.cos(rad_th) * kz) * np.abs(Ef + Eb)**2 /
                np.real(n_0 * np.cos(rad_th0)))
    elif pol == 'TM':
        absor = (np.imag(n * np.conj(np.cos(rad_th)) * 
                (kz * np.abs(Ef - Eb)**2 - np.conj(kz) * np.abs(Ef + Eb)**2)) /
                np.real(n_0 * np.conj(np.cos(rad_th0))))
    
    # Electric field
    if pol == 'TE':
        Ex = np.zeros(num_lambdas)
        Ey = Ef + Eb
        Ez = np.zeros(num_lambdas)
    elif pol == 'TM':
        Ex = (Ef - Eb) * np.cos(rad_th)
        Ey = np.zeros(num_lambdas)
        Ez = (-Ef - Eb) * np.sin(rad_th)
    
    return {
        'poyn': poyn,
        'absor': absor,
        'Ex': Ex,
        'Ey': Ey,
        'Ez': Ez
    }


def absorp_in_each_layer(coh_tmm_data):
    """
    Calculate absorption in each layer.
    
    Parameters:
    coh_tmm_data: output from coh_tmm()
    
    Returns:
    Array [wavelengths x layers] of absorption fractions
    """
    num_lambdas = len(coh_tmm_data['lam_vac'])
    num_layers = len(coh_tmm_data['d_list'])
    
    power_entering_each_layer = np.zeros((num_lambdas, num_layers))
    power_entering_each_layer[:, 0] = 1.0
    power_entering_each_layer[:, 1] = coh_tmm_data['power_entering']
    power_entering_each_layer[:, -1] = coh_tmm_data['T']
    
    for i in range(2, num_layers - 1):
        data = position_resolved(i, 0, coh_tmm_data)
        power_entering_each_layer[:, i] = data['poyn']
    
    final_answer = np.zeros((num_lambdas, num_layers))
    final_answer[:, :-1] = -np.diff(power_entering_each_layer, axis=1)
    final_answer[:, -1] = power_entering_each_layer[:, -1]
    final_answer[final_answer < np.finfo(float).eps] = 0
    
    return final_answer

print("Position-resolved and absorption functions defined successfully!")

Position-resolved and absorption functions defined successfully!


## Refractive Index Library

Load and manage refractive index data. You can either:
1. Upload an Excel file with refractive index data
2. Use predefined materials below

In [23]:
class RefractiveIndexLibrary:
    """Manage refractive index data for materials."""
    
    def __init__(self):
        self.data = None
        self.material_names = []
        
    def load_from_excel(self, filepath):
        """Load refractive index data from Excel file."""
        try:
            df = pd.read_excel(filepath)
            self.data = df
            
            # Extract material names (columns ending with _n)
            self.material_names = []
            for col in df.columns:
                if col.endswith('_n'):
                    mat_name = col[:-2]  # Remove '_n'
                    if f'{mat_name}_k' in df.columns:
                        self.material_names.append(mat_name)
            
            print(f"Loaded {len(self.material_names)} materials: {', '.join(self.material_names)}")
            return True
        except Exception as e:
            print(f"Error loading Excel file: {e}")
            return False
    
    def load_predefined(self):
        """Load some predefined materials for testing."""
        wavelengths = np.arange(300, 1000, 1)
        
        # Create a simple library with common materials
        data = {
            'Wavelength (nm)': wavelengths,
            'Air_n': np.ones_like(wavelengths, dtype=float),
            'Air_k': np.zeros_like(wavelengths, dtype=float),
            'Glass_n': np.ones_like(wavelengths, dtype=float) * 1.5,
            'Glass_k': np.zeros_like(wavelengths, dtype=float),
            'ITO_n': np.ones_like(wavelengths, dtype=float) * 2.0,
            'ITO_k': np.ones_like(wavelengths, dtype=float) * 0.01,
        }
        
        self.data = pd.DataFrame(data)
        self.material_names = ['Air', 'Glass', 'ITO']
        print(f"Loaded predefined materials: {', '.join(self.material_names)}")
        return True
    
    def get_refractive_index(self, material_name, wavelengths):
        """
        Get complex refractive index for a material at given wavelengths.
        
        Parameters:
        material_name: name of material
        wavelengths: array of wavelengths in nm
        
        Returns:
        Complex refractive index (n + ik)
        """
        if self.data is None:
            raise ValueError("No refractive index data loaded!")
        
        if material_name not in self.material_names:
            raise ValueError(f"Material '{material_name}' not found in library. Available: {self.material_names}")
        
        # Get wavelength column
        wl_data = self.data['Wavelength (nm)'].values
        n_data = self.data[f'{material_name}_n'].values
        k_data = self.data[f'{material_name}_k'].values
        
        # Interpolate to desired wavelengths
        n_interp = np.interp(wavelengths, wl_data, n_data)
        k_interp = np.interp(wavelengths, wl_data, k_data)
        
        return n_interp + 1j * k_interp
    
    def get_available_materials(self):
        """Return list of available materials."""
        return self.material_names.copy()


# Initialize the library
refr_index_lib = RefractiveIndexLibrary()

print("Refractive index library class defined!")

Refractive index library class defined!


## Load Your Refractive Index Data

Upload your Excel file or use predefined data:

In [24]:
# Option 1: Specify path to your Excel file
excel_file_path = 'Index_of_Refraction_library2.xls'  # Update this path

try:
    if refr_index_lib.load_from_excel(excel_file_path):
        print("✓ Excel file loaded successfully!")
    else:
        print("Using predefined materials instead...")
        refr_index_lib.load_predefined()
except:
    print("Excel file not found. Using predefined materials...")
    refr_index_lib.load_predefined()

# Display available materials
print(f"\nAvailable materials ({len(refr_index_lib.get_available_materials())}):")
for i, mat in enumerate(refr_index_lib.get_available_materials(), 1):
    print(f"  {i}. {mat}")

Loaded 83 materials: Air, Glass, absorber, Encapsulation, MgF2, ARC, aSi(i), aSi(n), aSi(p), cSi, Ag, ZAO, ZnO_refindexinfo, ZnO, Perov, Spiro, TiO2, Mo, Mo_refindexinfo, CdS, MoSe2, PDOT, Gold, Perov_tuned_1_6, Perov_tuned_1_65, Perov_tuned_1_7, Perov_tuned_1_75, Perov_tuned_1_8, Perov_tuned_1_85, Perov_tuned_1_9, Perov_tuned_1_95, Perov_tuned_2, Perov166, Perov169, Perov172, MAPbI3, Ca, PEDOT, PS, PSAnnealed, P3HT, PCBM, P3HTPCBMBlendDCB, NiO, NiOinhouse10, NiOinhouse50, NiOnewmodel, C60, MoO3, SnOinhouse, PTAA, Pero1.62Man18, PeroCs17Br17, Pero1.55Man18, PeroTCRao19, ITOlumtec, ITOlumtec2, ITOihteazhot, ITOihteazhot2, ITOfrontHol13, ITOihteazcold, ITOihteazcold2, ITO_front, ITO_rear, ITOsorizon, ITOfrontANU, ITOrearANU, ITOihteazHotCold, IZOt6x, IZOt11x, IZOt19x, IZOzsw, IOHzsw, SnO2npzsw, C60zsw, P508Absorberzsw, Spirozsw, MoO3zsw, 2.6, PMMA, MAPbI3-Paul, PbI2, CsCl
✓ Excel file loaded successfully!

Available materials (83):
  1. Air
  2. Glass
  3. absorber
  4. Encapsulation
  5

## Interactive Layer Stack Simulator

Configure your multilayer stack and calculate optical properties:

In [ ]:
class LayerStackSimulator:
    """Interactive simulator for multilayer optical stacks."""
    
    def __init__(self, refr_index_lib):
        self.refr_index_lib = refr_index_lib
        self.layers = []
        self.create_widgets()
        
    def create_widgets(self):
        """Create interactive widgets."""
        style = {'description_width': '150px'}
        
        # Wavelength range
        self.wl_min = widgets.IntText(value=400, description='Min λ (nm):', style=style)
        self.wl_max = widgets.IntText(value=900, description='Max λ (nm):', style=style)
        
        # Angle of incidence
        self.angle_inc = widgets.FloatSlider(value=0, min=0, max=85, step=1,
                                            description='Angle (°):', style=style)
        
        # Polarization
        self.polarization = widgets.Dropdown(options=['TE', 'TM'],
                                            value='TE', description='Polarization:', style=style)
        
        # Glass substrate correction
        self.glass_substrate = widgets.Checkbox(value=True, 
                                               description='Glass substrate correction (R+5%, T×95%)',
                                               style={'description_width': 'initial'})
        
        # Layer configuration
        self.num_layers = widgets.IntText(value=4, description='Number of layers:', 
                                         style=style, min=2, max=10)
        
        # Create initial layer widgets
        self.layer_widgets = []
        self.layer_container = widgets.VBox([])
        
        # Buttons
        self.update_layers_btn = widgets.Button(description='Update Layer Configuration',
                                               button_style='info')
        self.calculate_btn = widgets.Button(description='Calculate Optical Properties',
                                           button_style='success')
        
        # Output area
        self.output = widgets.Output()
        
        # Connect events
        self.update_layers_btn.on_click(self.update_layer_widgets)
        self.calculate_btn.on_click(self.calculate_and_plot)
        
        # Initial layer setup
        self.update_layer_widgets(None)
        
    def update_layer_widgets(self, b):
        """Update layer input widgets based on number of layers."""
        num = self.num_layers.value
        materials = self.refr_index_lib.get_available_materials()
        
        self.layer_widgets = []
        layer_boxes = []
        
        for i in range(num):
            # First and last layers should be semi-infinite
            if i == 0 or i == num - 1:
                thickness = widgets.FloatText(value=np.inf, description=f'Layer {i+1} thickness (nm):',
                                             disabled=True, style={'description_width': '180px'})
            else:
                thickness = widgets.FloatText(value=100, description=f'Layer {i+1} thickness (nm):',
                                             style={'description_width': '180px'})
            
            # Material selection
            default_mat = materials[min(i, len(materials)-1)]
            material = widgets.Dropdown(options=materials, value=default_mat,
                                       description=f'Layer {i+1} material:',
                                       style={'description_width': '180px'})
            
            layer_box = widgets.HBox([material, thickness])
            layer_boxes.append(layer_box)
            self.layer_widgets.append({'material': material, 'thickness': thickness})
        
        self.layer_container.children = layer_boxes
        
    def get_stack_configuration(self):
        """Extract current stack configuration."""
        stack_names = []
        thicknesses = []
        
        for layer in self.layer_widgets:
            stack_names.append(layer['material'].value)
            thicknesses.append(layer['thickness'].value)
        
        return stack_names, thicknesses
    
    def calculate_and_plot(self, b):
        """Run TMM calculation and plot results."""
        with self.output:
            clear_output(wait=True)
            
            try:
                # Get configuration
                stack_names, thicknesses = self.get_stack_configuration()
                lam_vac = np.arange(self.wl_min.value, self.wl_max.value + 1, 1)
                
                print(f"Calculating for stack: {' / '.join(stack_names)}")
                print(f"Thicknesses: {thicknesses}")
                
                # Build refractive index array
                n_array = np.zeros((len(lam_vac), len(stack_names)), dtype=complex)
                for i, mat_name in enumerate(stack_names):
                    n_array[:, i] = self.refr_index_lib.get_refractive_index(mat_name, lam_vac)
                
                # Run TMM
                d_list = np.array(thicknesses)
                coh_tmm_data = coh_tmm(self.polarization.value, n_array, d_list,
                                      self.angle_inc.value, lam_vac)
                
                # Get absorption in each layer
                A_layers = absorp_in_each_layer(coh_tmm_data)
                A_layers = np.clip(A_layers, 0, None)
                
                # Extract R, T, A
                R = coh_tmm_data['R']
                T = coh_tmm_data['T']
                A_total = 1 - R - T
                
                # Apply glass substrate correction if selected
                if self.glass_substrate.value:
                    R_corr = R + 0.05
                    T_corr = T * 0.95
                else:
                    R_corr = R
                    T_corr = T
                A_corr = 1 - R_corr - T_corr
                
                # Clip to physical bounds for plotting
                R_clip = np.clip(R_corr, 0, 1)
                T_clip = np.clip(T_corr, 0, 1)
                A_clip = np.clip(A_corr, 0, 1)
                
                # Calculate absorbance using clipped values
                denominator = 100 - R_clip * 100
                denominator[denominator == 0] = 1e-10
                absorbance = -np.log10(T_clip * 100 / denominator)
                
                # Plot results
                self.plot_results(lam_vac, R_clip, T_clip, A_clip, 
                                absorbance, A_layers, stack_names)
                
            except Exception as e:
                print(f"Error during calculation: {e}")
                import traceback
                traceback.print_exc()
    
    def plot_results(self, lam_vac, R, T, A, absorbance, A_layers, stack_names):
        """Plot optical properties using Plotly."""
        
        # Create subplots: 1 row x 1 col for R/T/A, then 2 separate plots below
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Optical Properties: Reflectance, Transmittance, Absorptance',
                          '', 'Absorbance', 'Absorption per Layer'),
            specs=[[{'colspan': 2}, None],
                   [{}, {}]],
            vertical_spacing=0.12,
            horizontal_spacing=0.1
        )
        
        # Plot 1: R, T, A (top, spanning both columns)
        fig.add_trace(
            go.Scatter(x=lam_vac, y=R, name='Reflectance (R)', 
                      line=dict(width=2, color='#1f77b4'),
                      mode='lines'),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=lam_vac, y=T, name='Transmittance (T)', 
                      line=dict(width=2, color='#ff7f0e'),
                      mode='lines'),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=lam_vac, y=A, name='Absorptance (A)', 
                      line=dict(width=2, color='#2ca02c'),
                      mode='lines'),
            row=1, col=1
        )
        
        # Plot 2: Absorbance (bottom left)
        fig.add_trace(
            go.Scatter(x=lam_vac, y=absorbance, name='Absorbance',
                      line=dict(width=2, color='purple'),
                      mode='lines', showlegend=False),
            row=2, col=1
        )
        
        # Plot 3: Absorption per layer (bottom right)
        colors = ['#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', '#aec7e8']
        for i in range(1, len(stack_names) - 1):  # Skip first and last (semi-infinite)
            color = colors[(i-1) % len(colors)]
            fig.add_trace(
                go.Scatter(x=lam_vac, y=A_layers[:, i], 
                          name=f'{stack_names[i]}',
                          line=dict(width=2, color=color),
                          mode='lines'),
                row=2, col=2
            )
        
        # Update axes labels and ranges
        fig.update_xaxes(title_text="Wavelength (nm)", row=1, col=1)
        fig.update_yaxes(title_text="R, T, A", range=[0, 1], row=1, col=1)
        
        fig.update_xaxes(title_text="Wavelength (nm)", row=2, col=1)
        fig.update_yaxes(title_text="Absorbance", row=2, col=1)
        
        fig.update_xaxes(title_text="Wavelength (nm)", row=2, col=2)
        ymax = np.max(A_layers[:, 1:-1]) if A_layers[:, 1:-1].size else 0
        fig.update_yaxes(title_text="Absorption", range=[0, max(1e-6, ymax)], row=2, col=2)
        
        # Update layout
        fig.update_layout(
            height=700,
            showlegend=True,
            hovermode='x unified',
            template='plotly_white',
            font=dict(size=11),
            margin=dict(l=50, r=50, t=80, b=50)
        )
        
        # Show the figure
        fig.show()
        
        # Print summary
        print("\n" + "="*60)
        print("SIMULATION SUMMARY")
        print("="*60)
        print(f"Wavelength range: {lam_vac[0]} - {lam_vac[-1]} nm")
        print(f"Angle of incidence: {self.angle_inc.value}°")
        print(f"Polarization: {self.polarization.value}")
        print(f"\nAverage values (across wavelength range):")
        print(f"  Reflectance:    {np.mean(R):.3f}")
        print(f"  Transmittance:  {np.mean(T):.3f}")
        print(f"  Absorptance:    {np.mean(A):.3f}")
        print("="*60)
    
    def display(self):
        """Display the interactive interface."""
        # Layout
        config_box = widgets.VBox([
            widgets.HTML("<h3>Simulation Parameters</h3>"),
            widgets.HBox([self.wl_min, self.wl_max]),
            widgets.HBox([self.angle_inc, self.polarization]),
            self.glass_substrate,
            widgets.HTML("<h3>Layer Stack Configuration</h3>"),
            widgets.HTML("<i>Note: First and last layers are semi-infinite (substrate/superstrate)</i>"),
            self.num_layers,
            self.update_layers_btn,
            self.layer_container,
            self.calculate_btn
        ])
        
        display(config_box, self.output)


# Create and display simulator
simulator = LayerStackSimulator(refr_index_lib)
print("Simulator created! Scroll down to see the interface.")

Simulator created! Scroll down to see the interface.


## Run the Simulator

Execute the cell below to display the interactive interface:

In [26]:
# Display the interactive simulator
simulator.display()

Output()

## Example Configurations

Here are some example layer stacks you can try (from the original MATLAB code):

### Example 1: Perovskite Solar Cell
- **Layer 1 (substrate):** Glass (inf)
- **Layer 2:** ITO (130 nm)
- **Layer 3:** CsCl (20 nm)
- **Layer 4:** MAPbI3-Paul (500 nm)
- **Layer 5 (air):** Air (inf)

### Example 2: Simple Perovskite on Glass
- **Layer 1:** Glass (inf)
- **Layer 2:** MAPbI3 (280 nm)
- **Layer 3:** Air (inf)

### Example 3: PMMA/Perovskite
- **Layer 1:** Air (inf)
- **Layer 2:** PMMA (100 nm)
- **Layer 3:** MAPbI3 (270 nm)
- **Layer 4:** Glass (inf)

---

## Notes

1. **Refractive Index Data:** Make sure your Excel file contains columns named like `MaterialName_n` and `MaterialName_k` for the real and imaginary parts of the refractive index.

2. **Glass Substrate Correction:** The checkbox applies the correction used in the original MATLAB code (R+5%, T×95%) to account for glass substrate effects.

3. **Wavelength Range:** Default is 400-900 nm, matching the original code. Adjust as needed.

4. **Angle of Incidence:** 0° = normal incidence. The code supports angles up to 85°.

5. **Polarization:** TE (s-polarized) or TM (p-polarized) light.

---

## Advanced: Manual Calculation Example

If you want to run calculations programmatically without the GUI:

In [27]:
# Example: Manual calculation without GUI
# Uncomment and modify as needed

"""
# Define parameters
lam_vac = np.arange(400, 900, 1)  # Wavelength range
pol = 'TE'  # Polarization
th_0 = 0  # Angle of incidence

# Define stack
stack_names = ['Glass', 'ITO', 'Glass']
thicknesses = [np.inf, 130, np.inf]

# Build refractive index array
n_array = np.zeros((len(lam_vac), len(stack_names)), dtype=complex)
for i, mat_name in enumerate(stack_names):
    n_array[:, i] = refr_index_lib.get_refractive_index(mat_name, lam_vac)

# Run TMM
d_list = np.array(thicknesses)
coh_tmm_data = coh_tmm(pol, n_array, d_list, th_0, lam_vac)

# Extract results
R = coh_tmm_data['R']
T = coh_tmm_data['T']
A = 1 - R - T

# Plot with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=lam_vac, y=R, name='R', mode='lines', line=dict(width=2)))
fig.add_trace(go.Scatter(x=lam_vac, y=T, name='T', mode='lines', line=dict(width=2)))
fig.add_trace(go.Scatter(x=lam_vac, y=A, name='A', mode='lines', line=dict(width=2)))
fig.update_layout(
    xaxis_title='Wavelength (nm)',
    yaxis_title='R, T, A',
    template='plotly_white',
    hovermode='x unified'
)
fig.show()
"""

print("Manual calculation example provided (commented out).")

Manual calculation example provided (commented out).
